## Parse and translate LSARS data ##

In [1]:
import json
import pandas as pd
from tqdm import tqdm

from deep_translator import GoogleTranslator, DeeplTranslator
from google_trans_new import google_translator  
import googletrans
from pygoogletranslation import Translator

In [2]:
data_path = "../data/chinese_data/"
training_data_path = data_path + "train.json"
test_data_path = data_path + "test.json"

deepl_api_key = "f6b7310c-7474-a87e-a191-27bc9763eb6c:fx"

In [3]:
test_jsons = []

with open(test_data_path, 'r') as fp:
  for line in fp:
      test_jsons.append(json.loads(line))

In [4]:
test_jsons[0].keys()

dict_keys(['item_id', 'lq_tokens_list', 'lq_aspect_list', 'hq_tokens', 'hq_aspect'])

In [21]:
def join_tokens(token_list):
    return "".join(token_list)

def process_json(json):
    item_id = json["item_id"]
    
    summary = join_tokens(json["hq_tokens"])
    reviews = json["lq_tokens_list"]
    review_str = ""
    reviews = [join_tokens(review) for review in reviews]

    translated_reviews = GoogleTranslator('zh-CN', 'en').translate_batch(reviews)
    translated_summary = GoogleTranslator(source='chinese (simplified)', target='en').translate(text=summary)

    return item_id, translated_reviews, translated_summary

#id, reviews, summary = process_json(test_jsons[0])

In [18]:
pd.DataFrame([[id, reviews, summary]], columns = ["id","reviews", "summary"])

,id,reviews,summary
0,19102166197,[The baby was received. I really didn’t expect...,The actual baby is the same as the picture. Th...


In [7]:
len(test_jsons)

2938

In [22]:
ids, reviews, summaries = [], [], []

for json in tqdm(test_jsons[0:100]):

    item_id, translated_reviews, translated_summary = process_json(json)
    ids.append(item_id)
    reviews.append(translated_reviews)
    summaries.append(translated_summary)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [49:24<00:00, 29.65s/it]


In [23]:
summaries

['The actual baby is the same as the picture. The style and color matching are very coordinated. Against the background of the blue sea water, it looks more fresh and beautiful. The size is also very standard. After the skirt was put into the water, it was slightly discolored, but there was no shrinkage. This is very important. A little, otherwise all the efforts will be wasted. The product packaging is very good and the logistics is fast. Full praise.',
 "The pants are of good quality. I bought them at a discount on Queen's Day. They are very elastic and the design is very fashionable. Everyone must have a pair of versatile pants.",
 '165100 pounds, size M is just right for wearing long johns. The pants are very soft and comfortable. The quality is good and the velvet is warm, but a little sticky. But this kind of black pants should be more or less suitable.',
 "It's the same as tailor-made, I'm 158, the customer service recommended size M, it fits very well, and it looks very elegant

In [24]:
result_df = pd.DataFrame({"id": ids, "reviews": reviews, "summary": summaries})
result_df

,id,reviews,summary
0,19102166197,[The baby was received. I really didn’t expect...,The actual baby is the same as the picture. Th...
1,529618039778,[They are perfect. It is very difficult for a ...,The pants are of good quality. I bought them a...
2,535894028777,[I gave them to a friend's child. They are nic...,"165100 pounds, size M is just right for wearin..."
3,539394568874,[My first reaction after receiving it and wear...,"It's the same as tailor-made, I'm 158, the cus..."
4,540381936384,"[Very beautiful, just like the description, th...","The quality is very good, the actual product i..."
...,...,...,...
95,560272505619,[I like the upper body very much. I prefer a s...,I like the clothes very much. The quality is v...
96,560275253291,[These pants are really good-looking. I was wo...,"The price is very worth it, the fabric is very..."
97,560617877003,"[The clothes are very nice, and I look pretty ...",The color is very beautiful. The more I look a...
98,560643173236,"[This color is called skin!, I put the clothes...",Very satisfied. I am a dark person and this co...


In [25]:
result_df["review_str"] = result_df["reviews"].apply(lambda x: "|||||".join(x))
result_df

,id,reviews,summary,review_str
0,19102166197,[The baby was received. I really didn’t expect...,The actual baby is the same as the picture. Th...,The baby was received. I really didn’t expect ...
1,529618039778,[They are perfect. It is very difficult for a ...,The pants are of good quality. I bought them a...,They are perfect. It is very difficult for a s...
2,535894028777,[I gave them to a friend's child. They are nic...,"165100 pounds, size M is just right for wearin...",I gave them to a friend's child. They are nice...
3,539394568874,[My first reaction after receiving it and wear...,"It's the same as tailor-made, I'm 158, the cus...",My first reaction after receiving it and weari...
4,540381936384,"[Very beautiful, just like the description, th...","The quality is very good, the actual product i...","Very beautiful, just like the description, the..."
...,...,...,...,...
95,560272505619,[I like the upper body very much. I prefer a s...,I like the clothes very much. The quality is v...,I like the upper body very much. I prefer a sl...
96,560275253291,[These pants are really good-looking. I was wo...,"The price is very worth it, the fabric is very...",These pants are really good-looking. I was wor...
97,560617877003,"[The clothes are very nice, and I look pretty ...",The color is very beautiful. The more I look a...,"The clothes are very nice, and I look pretty w..."
98,560643173236,"[This color is called skin!, I put the clothes...",Very satisfied. I am a dark person and this co...,This color is called skin!|||||I put the cloth...


In [28]:
result_df = result_df[["id", "summary", "review_str"]]
result_df

,id,summary,review_str
0,19102166197,The actual baby is the same as the picture. Th...,The baby was received. I really didn’t expect ...
1,529618039778,The pants are of good quality. I bought them a...,They are perfect. It is very difficult for a s...
2,535894028777,"165100 pounds, size M is just right for wearin...",I gave them to a friend's child. They are nice...
3,539394568874,"It's the same as tailor-made, I'm 158, the cus...",My first reaction after receiving it and weari...
4,540381936384,"The quality is very good, the actual product i...","Very beautiful, just like the description, the..."
...,...,...,...
95,560272505619,I like the clothes very much. The quality is v...,I like the upper body very much. I prefer a sl...
96,560275253291,"The price is very worth it, the fabric is very...",These pants are really good-looking. I was wor...
97,560617877003,The color is very beautiful. The more I look a...,"The clothes are very nice, and I look pretty w..."
98,560643173236,Very satisfied. I am a dark person and this co...,This color is called skin!|||||I put the cloth...


In [30]:
result_df.to_csv("../data/translated_data/test_trans_0-100.csv", index=False)